# Playground

In [8]:
import torch
from pathlib import Path

from uda import UNetConfig, UNet

config_dir = Path("../config")

In [9]:
config = UNetConfig.from_file(config_dir / "unet.yaml")
model = UNet(config)

n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
n_params

[128, 64, 64]
[64, 32, 32]
[32, 16, 16]


482336

In [10]:
x = torch.empty(4, 1, 64, 64).normal_()
model(x)

3
3
torch.Size([4, 128, 8, 8])
torch.Size([4, 64, 16, 16])
torch.Size([4, 32, 32, 32])
torch.Size([4, 16, 64, 64])


tensor([[[[ 2.7318, -0.6117,  1.7253,  ...,  1.2500,  4.4189,  1.6141],
          [ 5.5466,  2.6231,  5.3372,  ...,  4.3573,  2.1823,  2.8115],
          [-0.3926,  1.5672,  2.6727,  ..., -0.0872,  5.6519,  3.2005],
          ...,
          [ 1.9482,  6.9440,  1.9795,  ..., -0.9228, -0.0685,  2.4322],
          [ 1.9360,  0.4992,  4.1460,  ...,  4.0710,  1.1738, -0.3389],
          [ 4.0056,  2.4173,  1.5333,  ...,  3.0330,  1.2237,  1.4150]]],


        [[[ 2.4713,  1.3349,  1.5472,  ...,  0.0855,  2.1126,  0.7644],
          [ 2.7151,  3.7070,  7.6580,  ..., -3.3017,  2.9902,  3.1128],
          [ 8.2186, -4.4179, -1.4436,  ...,  3.9846,  7.3984,  1.4928],
          ...,
          [ 4.2528,  1.4439,  1.8627,  ..., -1.6131,  1.1575,  5.1853],
          [ 0.9547,  1.8905,  1.9436,  ...,  5.0177,  0.7633, -0.0404],
          [ 4.9619,  2.8339,  1.4942,  ...,  2.9976,  1.0259,  2.0464]]],


        [[[ 1.7771, -0.6669,  2.1898,  ...,  1.4632,  2.5689, -0.1638],
          [ 3.8063,  4.330